In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread

In [2]:
my_data= 'data'
os.listdir(my_data) 

['train_tomato', 'valid_tomato']

In [3]:
train_path = my_data+'/train_tomato/'
test_path = my_data+'/valid_tomato/'

In [4]:
classes =['Tomato___Bacterial_spot',
          'Tomato___Early_blight',
          'Tomato___healthy',
          'Tomato___Late_blight',
          'Tomato___Leaf_Mold',
          'Tomato___Septoria_leaf_spot',
          'Tomato___Spider_mites Two-spotted_spider_mite',
          'Tomato___Target_Spot',
          'Tomato___Tomato_mosaic_virus',
          'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [5]:
path='data/train_tomato'
for tomato_class in classes:
    class_path = os.path.join(path, tomato_class)
    max_values = []
    min_values = []
    for image in os.listdir(class_path):
        img = imread(os.path.join(class_path, image))
        max_val = img.max()
        min_val = img.min()
        max_values.append(max_val)
        min_values.append(min_val)
    max_class_val = max(max_values)
    min_class_val = min(min_values)
    print("***************************")
    print(f"Class: {tomato_class}")
    print(f"Max value: {max_class_val}")
    print(f"Min value: {min_class_val}")

print("***************************")

***************************
Class: Tomato___Bacterial_spot
Max value: 255
Min value: 0
***************************
Class: Tomato___Early_blight
Max value: 255
Min value: 0
***************************
Class: Tomato___healthy
Max value: 255
Min value: 0
***************************
Class: Tomato___Late_blight
Max value: 255
Min value: 0
***************************
Class: Tomato___Leaf_Mold
Max value: 255
Min value: 0
***************************
Class: Tomato___Septoria_leaf_spot
Max value: 255
Min value: 0
***************************
Class: Tomato___Spider_mites Two-spotted_spider_mite
Max value: 255
Min value: 0
***************************
Class: Tomato___Target_Spot
Max value: 255
Min value: 0
***************************
Class: Tomato___Tomato_mosaic_virus
Max value: 255
Min value: 0
***************************
Class: Tomato___Tomato_Yellow_Leaf_Curl_Virus
Max value: 255
Min value: 0
***************************


In [6]:
input_shape = (256,256)

In [7]:
batch_size = 64

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

Found 18345 files belonging to 10 classes.
Using 14676 files for training.


In [9]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=input_shape,
  batch_size=batch_size)

Found 18345 files belonging to 10 classes.
Using 3669 files for validation.


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_ds = train_ds.map(lambda x,y: (x/255, y))
val_ds = val_ds.map(lambda x,y: (x/255, y))

In [12]:
image_batch, labels_batch = next(iter(train_ds))

In [13]:
first_image = image_batch[0]

In [14]:
print(np.min(first_image), np.max(first_image))

0.0 0.7882353


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,BatchNormalization

In [16]:
input_shape =  (256,256,3) 

In [17]:
num_classes = len(classes)
num_classes

10

In [18]:
from tensorflow.keras.applications import MobileNetV2

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
early_stop = EarlyStopping(monitor='val_loss',patience=20)

In [21]:
from tensorflow.keras.optimizers import Adam

In [22]:
MobileNetV2_model = Sequential()

pretrained_model = MobileNetV2(include_top=False,
                   input_shape=(256, 256, 3),
                   pooling='avg',
                   weights='imagenet')

for layer in pretrained_model.layers:
    layer.trainable = False

MobileNetV2_model.add(pretrained_model)
MobileNetV2_model.add(Flatten())
# ANN
MobileNetV2_model.add(Dense(512, activation='relu'))
MobileNetV2_model.add(Dense(256, activation='relu'))
MobileNetV2_model.add(Dropout(0.2))

MobileNetV2_model.add(Dense(32, activation='relu'))



MobileNetV2_model.add(Dense(num_classes, activation='softmax'))

MobileNetV2_model.compile(optimizer=Adam(learning_rate=0.003),
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                          metrics=['accuracy'])

MobileNetV2_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 1280)             2257984   
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                8224      
                                                        

In [23]:
MobileNetV2_model_results = MobileNetV2_model.fit(train_ds, epochs=50,
                             validation_data=val_ds,
                             callbacks=[early_stop])

Epoch 1/50
230/230 [==============================] - 65s 226ms/step - loss: 1.0024 - accuracy: 0.6466 - val_loss: 0.4823 - val_accuracy: 0.8384
Epoch 2/50
230/230 [==============================] - 48s 204ms/step - loss: 0.3860 - accuracy: 0.8679 - val_loss: 0.3848 - val_accuracy: 0.8744
Epoch 3/50
230/230 [==============================] - 48s 205ms/step - loss: 0.2551 - accuracy: 0.9143 - val_loss: 0.3592 - val_accuracy: 0.8795
Epoch 4/50
230/230 [==============================] - 47s 205ms/step - loss: 0.2045 - accuracy: 0.9313 - val_loss: 0.3008 - val_accuracy: 0.9060
Epoch 5/50
230/230 [==============================] - 47s 206ms/step - loss: 0.1849 - accuracy: 0.9385 - val_loss: 0.2339 - val_accuracy: 0.9207
Epoch 6/50
230/230 [==============================] - 48s 207ms/step - loss: 0.1451 - accuracy: 0.9537 - val_loss: 0.2272 - val_accuracy: 0.9250
Epoch 7/50
230/230 [==============================] - 48s 206ms/step - loss: 0.1198 - accuracy: 0.9606 - val_loss: 0.2593 - val_ac

In [24]:
# Model Evaluation
val_loss,val_acc = MobileNetV2_model.evaluate(val_ds)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

58/58 [==============================] - 8s 143ms/step - loss: 0.5118 - accuracy: 0.9092
Validation Loss: 0.5118280053138733
Validation Accuracy: 0.9092395901679993


In [25]:
MobileNetV2_model.save('tomato_disease_model.h5')